## 任务说明

本项目实现了基于编解码框架方法的图像描述生成任务，可以自动为图片生成流畅关联的自然语言描述。

总共实现了三种不同的模型：

- 1.CNN/ViT与GRU结合
- 2.基于自注意力的网格/区域表示
- 3.视觉Transformer+Transformer解码器

实验使用的数据集采用了deepfashion-multimodal。

通过实现三项评测标准作为模型性能评估的依据：

- 1.ROUGE-L
- 2.CIDEr-D
- 3.METEOR

通过实现不同的模型结构和评测标准，我们旨在探索和评估各种方法在图像描述领域的有效性和实用性。

最后，我们考虑通过模型微调、数据集扩展等方式提升模型系统的性能。

（待补充）

## 实验数据


### 数据集下载
- 数据集使用的是Deepfashion-multimodal([下载地址](https://github.com/yumingj/DeepFashion-MultiModal)).
- 数据集包含12,694张服饰图片，每个图片有若干条句子描述.
- 数据集被划分为20%的测试集与80%的训练集.
- 下载解压会，将图片放在指定目录(本代码中将该目录设置为../../data/deepfashion/)下的images文件夹里.



### 数据预处理

#### 预处理步骤：

#### 代码：


## 实验环境
(待补充)

## 所用的方法或模型

### 评测指标

#### ROUGE-L (Recall-Oriented Understudy for Gisting Evaluation)
- ROUGE-L是基于Rouge的评估自动文摘以及机器翻译的指标，特别关注于生成文本的长距离依赖和结构，计算利用了最长公共子序列。
最长公共子序列的召回和精度:
$$
R_{lcs} = \frac{LCS(X, Y)}{m}
,
P_{lcs} = \frac{LCS(X, Y)}{n}
$$

 Rouge-L指标计算公式为：
$$
F_{lcs} = \frac{(1 + \beta^2)R_{lcs}P_{lcs}}{R_{lcs} + \beta^2 P_{lcs}}
$$

 - X表示标准结果.
 - Y表示模型生成结果
 - m表示X的长度.
 - n表示Y的长度.
 - LSC(X,Y)表示X,Y的最长公共子序列.
 - 超参数β通常设置较大.
 - $R_{lcs}$表示召回率.
 - $p_{lcs}$表示准确率.

#### CIDEr-D  (Consensus-based Image Description Evaluation)
- CIDEr-D基于CIDEr，是一种专门面向于图像描述问题的指标，通过基于候选和参考句子长度之间差异的高斯惩罚与计数上的限制来使得评判标准更符合人类标准：

对于n元组$ω_k$，出现在某个参考语句$(ref)_{ij}$中的次数记为$h_k(s_{ij})$，出现在某个候选语句$(can)c_i$中的次数记为$c_i$，因此这个n元组的TF-IDF表示为：
$$
g_k(s_{ij}) = \frac{h_k(s_{ij})}{\sum_{\omega_l \in \Omega} h_k(s_{ij})} \log\left(\frac{|I|}{\sum_{l_p \in I} \min(1, \sum_{q} h_k(s_{pq}))}\right)^k
$$

- 式中$\omega$表示所有的n元组的个数.
- I表示所有的图片个数。

$CIDEr_n$则是用来计算候选语句和参考语句在n长度的元组表示时的相似度，用公式表示为:
$$
CIDEr_n(c_i, S_i) = \frac{1}{m} \sum_j \frac{g^n(c_i) \cdot g^n(s_{ij})}{\|g^n(c_i)\| \cdot \|g^n(s_{ij})\|}
$$
对所有的$CIDEr_n$取平均得到CIDEr:
$$
CIDEr(c_i, S_i) = \sum_{n=1}^{N} \omega_n CIDEr_n(c_i, S_i)
$$
- CIDEr-D改进的目的是为了防止评价指标的gaming问题，首先，去除了将句子中的单词转换为原始形式这一步，让句子生成时对应的状态是对的。

在CIDEr评价指标中，对一些重复单词得到的长句子，CIDEr的分数偏高，为了减少这种影响，文章加入了高斯惩罚项：
$$
CIDEr-D_n(c_i, S_i) = \frac{10}{m} \sum_j e^{-\frac{(l(c_i) - l(s_{ij}))^2}{2\sigma^2}} \frac{\min(g^n(c_i), g^n(s_{ij})) \cdot g^n(s_{ij})}{\|g^n(c_i)\| \cdot \|g^n(s_{ij})\|}
$$
- 式中，$l(c_i)$和$l(s_ij)$表示candidate和reference sentences的长度，10是为了将CIDEr-D的值和其它评价指标的值大小接近。

最终的CIDEr-D的计算公式为:
$$
CIDEr - D(c_i, S_i) = \frac{1}{N} \sum_{n=1}^{N} CIDEr - D_n(c_i, S_i)
$$


#### METEOR
- Meteor关注到那些翻译准确，但是与候选译文对不上的参考译文，并考虑了基于整个语料库上的准确率和召回率，利用二者的调和平均值来作为评判标准:
$$
F\ - Score = \frac{P \cdot R}{\alpha \cdot P + (1 - \alpha) \cdot R}
$$
- 对于词与词之间的顺序问题，该指标引入了fragmentation penalty，如果两句子中相互匹配的单词是相邻的，那么就定义为同一块（chunk），块的总数ch，frag定义为$frag=ch/m$ 。

因此惩罚因子定义为:

$$
Pen = \gamma \cdot frag^\beta
$$
- 其中$0<\gamma<1$，代表惩罚的力度。

最终的Meteor的计算方式为：
$$
METEOR = (1 - Pen) \cdot F\ - Score
$$

#### 实现代码
(放这里或者分别放在每个指标下面)

### 模型

#### CNN/ViT与GRU结合



#### 基于自注意力的网格/区域表示



#### 视觉Transformer+Transformer解码器



## 实验结果
（待补充）


## 实验结果分析


## 总结